# Data Preparation

I need to prepare data for my models to work so I plan on doing that here within this notebook.

In [2]:
# Pandas is a huge tool for this stuff
import pandas as pd

## Visualization

My first step here is to get an idea of the shape of the data I am working with.

In [7]:
# this ran for like 4 minutes without doing anything
# frame = pd.read_csv('us-stock-fundamentals.csv', header=None, error_bad_lines=False)

# there's a possibility the files are too large to do anything worthwhile with. I'll try taking a chunk.
frame = pd.read_csv('us-stock-fundamentals.csv', on_bad_lines='skip', sep=';')

frame

,Indicator,Quarter,Latest NAICS Industry Sector Name,Latest Name,SEC ID,Value
0,Comprehensive Income Net Of Tax,2012/Q4,Administrative and Support and Waste Managemen...,"Nxt-Id, Inc.",1566826,NaN
1,Comprehensive Income Net Of Tax,2012/Q3,Administrative and Support and Waste Managemen...,"Nxt-Id, Inc.",1566826,NaN
2,Comprehensive Income Net Of Tax,2012/Q1,Administrative and Support and Waste Managemen...,"Nxt-Id, Inc.",1566826,NaN
3,Comprehensive Income Net Of Tax,2013/Q3,Administrative and Support and Waste Managemen...,"Nxt-Id, Inc.",1566826,NaN
4,Comprehensive Income Net Of Tax,2011/Q2,Administrative and Support and Waste Managemen...,"Nxt-Id, Inc.",1566826,NaN
...,...,...,...,...,...,...
3549953,Weighted Average Number Of Diluted Shares Outs...,2011/Q1,Manufacturing,Terra Tech Corp.,1451512,NaN
3549954,Weighted Average Number Of Diluted Shares Outs...,2014/Q1,Manufacturing,Technical Communications Corp,96699,1838907.0
3549955,Weighted Average Number Of Diluted Shares Outs...,2015/Q1,Manufacturing,Technical Communications Corp,96699,1846399.0
3549956,Weighted Average Number Of Diluted Shares Outs...,2015/Q2,Manufacturing,Technical Communications Corp,96699,1839520.0


## US-Stock-Fundamentals

Actually seems to have very little useful data for us to work with, but it is interesting that between this and the othe CSV we were provided, ids seem to be common. The **SEC ID** for a stock seems to be a regular representative of a company which will be nice when we need to corroborate or consolidate data between companies. It also seems to be the case that the time frame over which we are going to have access to these fundamental indicators is once a quarter, which is about 4 times a year, pretty self explanatorily. 

In [8]:
frame = pd.read_csv('indicators_by_company.csv')

frame

,company_id,indicator_id,2010,2011,2012,2013,2014,2015,2016
0,1000045,AccountsPayableAndAccruedLiabilitiesCurrentAnd...,NaN,6612429.0,7405579.0,8924919.0,7841070.0,5839000.0,NaN
1,1000045,AccumulatedDepreciationDepletionAndAmortizatio...,NaN,NaN,2111343.0,2242703.0,2236449.0,2462000.0,NaN
2,1000045,AdjustmentForAmortization,NaN,NaN,-11482251.0,-13490892.0,-13852305.0,-13811000.0,NaN
3,1000045,Assets,NaN,257236034.0,263835468.0,283429579.0,302528591.0,325309000.0,NaN
4,1000045,AssetsHeldForSaleAtCarryingValue,NaN,1373001.0,1203664.0,1696330.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1907878,9984,UnrecognizedTaxBenefitsInterestOnIncomeTaxesAc...,NaN,0.0,0.0,1031000.0,1031000.0,1923000.0,NaN
1907879,9984,UnrecognizedTaxBenefitsReductionsResultingFrom...,NaN,177000.0,0.0,0.0,NaN,215000.0,NaN
1907880,9984,ValuationAllowanceDeferredTaxAssetChangeInAmount,NaN,-13355000.0,8255000.0,-6063000.0,NaN,NaN,NaN
1907881,9984,WeightedAverageNumberOfDilutedSharesOutstanding,NaN,55931882.0,55224457.0,54973344.0,55723267.0,55513219.0,NaN


## Indicators-By-Company

This is a nicer data set that seems to come with a more useful amount of information. The shape is interesting, however. To make it work nicer with something like pandas, it's nice that it has 9 columns, but that leaves us with multiple rows for single companies. These different rows represent different indicators. It's also worth mentioning that this data is yearly now and seems to span 5 years at best fo r alot of these companies. 

It might be worthwhile to look in to how many of these indicators exist for different companies.

In [16]:
# luckily, single companies seem to be grouped by their ids which makes this a little easier to do
indicator_counts = dict()

# I want to see what the indicator numbers look like for each one of these companies throughout the entire thing.
unique_inds = frame['indicator_id'].unique()

for ind in range(len(unique_inds)):
    print('{:.2f}%'.format(ind / len(unique_inds) * 100), end='\r')
    indicator_counts[unique_inds[ind]] = len(frame[frame['indicator_id'] == unique_inds[ind]])


In [28]:
# it may be useful to have a sorted array of the indicators by the amount of data points we have for them

count = [(-count, val) for val, count in indicator_counts.items()]
count.sort()

indicators_by_name = list(indicator_counts.keys())
indicators_by_name.sort()

indicators = open("Indicator-Names.txt", "w")

for ind in indicators_by_name:
    indicators.write(ind + '\n')

indicators.close()

## Indicators

I need to find the data indicators that are of particular interest to me. These are values that can be used to calculate heuristics for fundamental analysis to plug into the system _and_ have high enough counts to present a large enough training set for the data I am using. 

Soon I may have to decide what to do about incomplete records and how I will treat them within the data set I am building, but for now I need to make sure that will be enough of an issue later on.

---
### Indicator-Names.txt
This is an alphabetical list of all of the indicators present within the data set I am currently looking at.

## Important Indicators

Here I will keep a list of indicators of interest to me and their associated calculations they may be pertinent to.

|Indicator|Indicator Count|
|--|--|
|EarningsPerShareBasic|5501|
|EarningsPerShareDiluted|5084|

In [54]:
print(indicator_counts['EarningsPerShareBasic'])
print(indicator_counts['EarningsPerShareDiluted'])
print(indicator_counts['SharesIssuedPricePerShare'])
print(indicator_counts['FairValueInputsLongTermRevenueGrowthRate'])
print(indicator_counts['CommonStockSharesOutstanding'])


print(count[:40])

5501
5084
182
44
7571
[(-9469, 'LiabilitiesAndStockholdersEquity'), (-9460, 'Assets'), (-8926, 'NetIncomeLoss'), (-8843, 'StockholdersEquity'), (-8735, 'CashAndCashEquivalentsAtCarryingValue'), (-8412, 'CashAndCashEquivalentsPeriodIncreaseDecrease'), (-8374, 'RetainedEarningsAccumulatedDeficit'), (-8303, 'CommonStockSharesAuthorized'), (-8192, 'NetCashProvidedByUsedInOperatingActivities'), (-8093, 'CommonStockValue'), (-8092, 'CommonStockSharesIssued'), (-8078, 'NetCashProvidedByUsedInFinancingActivities'), (-7848, 'CommonStockParOrStatedValuePerShare'), (-7571, 'CommonStockSharesOutstanding'), (-7512, 'Liabilities'), (-7435, 'LiabilitiesCurrent'), (-7432, 'AssetsCurrent'), (-7430, 'PropertyPlantAndEquipmentNet'), (-7412, 'NetCashProvidedByUsedInInvestingActivities'), (-7205, 'OperatingIncomeLoss'), (-7031, 'IncomeTaxExpenseBenefit'), (-6738, 'InterestExpense'), (-6551, 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment'), (-6484, 'ShareBasedCompensation'), (-603

In [ ]:
frame[(frame['indicator_id'] == 'AssetsCurrent')]

In [71]:
frame[(frame['indicator_id'] == 'StockholdersEquity')]

,company_id,indicator_id,2010,2011,2012,2013,2014,2015,2016
116,1000045,StockholdersEquity,NaN,NaN,1.269651e+08,1.419376e+08,8.988794e+07,1.028490e+08,NaN
499,1000180,StockholdersEquity,5.782624e+09,7.064358e+09,7.263901e+09,6.967872e+09,6.528059e+09,5.738924e+09,NaN
796,1000228,StockholdersEquity,NaN,2.432222e+09,2.613585e+09,2.785197e+09,2.813594e+09,2.884256e+09,NaN
1133,1000229,StockholdersEquity,NaN,1.779030e+08,1.822300e+08,1.633230e+08,8.757300e+07,-2.906400e+07,NaN
1357,1000230,StockholdersEquity,NaN,2.820917e+07,3.064436e+07,3.019867e+07,3.100657e+07,2.663061e+07,NaN
...,...,...,...,...,...,...,...,...,...
1906624,99302,StockholdersEquity,NaN,2.737800e+07,3.165000e+07,3.008300e+07,3.431800e+07,3.891100e+07,NaN
1906835,99359,StockholdersEquity,NaN,3.815200e+07,4.307200e+07,5.048400e+07,6.711500e+07,NaN,NaN
1907168,99771,StockholdersEquity,NaN,1.179690e+08,NaN,8.871000e+07,8.100300e+07,NaN,NaN
1907596,99780,StockholdersEquity,NaN,1.863800e+09,2.053000e+09,2.402100e+09,2.995900e+09,3.653900e+09,NaN


In [72]:
frame[(frame['indicator_id'] == 'WeightedAverageNumberOfSharesOutstandingBasic')]

,company_id,indicator_id,2010,2011,2012,2013,2014,2015,2016
123,1000045,WeightedAverageNumberOfSharesOutstandingBasic,NaN,NaN,11977174.0,12096000.0,12012765.0,7622000.0,NaN
521,1000180,WeightedAverageNumberOfSharesOutstandingBasic,232531000.0,239484000.0,242076000.0,234886000.0,222714000.0,205443000.0,NaN
829,1000228,WeightedAverageNumberOfSharesOutstandingBasic,NaN,90120000.0,87499000.0,85926000.0,84265000.0,82844000.0,NaN
1152,1000229,WeightedAverageNumberOfSharesOutstandingBasic,NaN,46286000.0,47211000.0,45692000.0,44362000.0,42747000.0,NaN
1370,1000230,WeightedAverageNumberOfSharesOutstandingBasic,NaN,NaN,6455817.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1906635,99302,WeightedAverageNumberOfSharesOutstandingBasic,NaN,7309000.0,7404000.0,7080000.0,6798000.0,6887000.0,NaN
1906845,99359,WeightedAverageNumberOfSharesOutstandingBasic,NaN,9473000.0,9511000.0,9643000.0,9778000.0,NaN,NaN
1907200,99771,WeightedAverageNumberOfSharesOutstandingBasic,NaN,NaN,NaN,6449726.0,6452557.0,NaN,NaN
1907617,99780,WeightedAverageNumberOfSharesOutstandingBasic,NaN,77500000.0,77300000.0,76400000.0,151000000.0,150200000.0,NaN


In [73]:
1.779030e+08	/ 46286000.0

3.8435596076567426